**Códigos utilizados** 

In [1]:
import Bio 
from Bio import Entrez
import re

In [9]:
def download_pubmed(keyword):
    """La primera función download_pubmed se usa para extraer el conteo de los artículos con una palabra clave -keyword- definida respectivamente"""
    Entrez.email = "jessica.verdezoto@est.ikiam.edu.ec"
    hd1 = Entrez.read(Entrez.esearch(db="pubmed",
                        term=keyword,
                        usehistory="y"))
    webenv = record["WebEnv"]
    query_key = record["QueryKey"]
    hd1 = Entrez.efetch(db="pubmed",
                    retstart =0,
                    rettype="medline",
                    retmode="text",
                    retmax=543,webenv=webenv, query_key=query_key)
    out_hd1 = open("keyword.txt", "w")
    data = hd1.read()
    datos = re.sub(r'\n\s{6}', '', data)
    return datos


In [11]:
def mining_pubs(tipo):
    """La segunda función mining_pubs se usa para extraer datos mas precisos ubicados en el texto, utilizando otros parametros, en este caso usamos -tipo-"""
    * Si el tipo es "DP" proporciona el año de publicación del artículo. Retorna como un *dataframe* con el **PMID** y el **DP_year**.
    * Si el tipo es "AU" proporciona el número de autores por **PMID**. Retorna como un *dataframe* con el **PMID** y el **num_auth**. 
    * Si el tipo es "AD" proporcion el conteo de autores por país. El retorno es un *dataframe* con el **country** y el **num_auth**.
    
    pass
    return

SyntaxError: invalid syntax (2244992499.py, line 3)